# 极客时间 - 程序员基础数学课 - [第37课 - 矩阵计算PageRank](https://time.geekbang.org/column/article/85194)

首先，我使用 sklearn 库中的 CountVectorizer，对一个测试的文档集合构建特征，也就是词典。这个测试集合有 7 句话，2 句关于篮球，2 句关于电影，还有 3 句关于游戏。具体代码如下：

In [7]:
import numpy as np

# 设置确定随机跳转概率的alpha、网页结点数
alpha = 0.9
N = 5

# 初始化随机跳转概率的矩阵
jump = np.full([2,1], [[alpha], [1-alpha]], dtype=float)

print("随机跳转概率: ")
print(jump)

# 邻接矩阵的构建
adj = np.full([N,N], [[0,0,1,0,0],[1,0,1,0,0],[1,0,0,0,0],[0,0,0,0,0],[0,1,0,0,0]], dtype=float)

print("邻接矩阵: ")
print(adj)

# 对邻接矩阵进行归一化
row_sums = adj.sum(axis=1)      # 对每一行求和
print("row_sums: ")
print(row_sums)

row_sums[row_sums == 0] = 0.1   # 防止由于分母出现0而导致的Nan
print("row_sums处理0->0.1: ")
print(row_sums)

print("row_sums[:, np.newaxis]: ")
print(row_sums[:, np.newaxis])

adj = adj / row_sums[:, np.newaxis] # 除以每行之和的归一化
print("邻接矩阵： ")
print(adj)

# 初始的PageRank值，通常是设置所有值为1.0
pr = np.full([1,N], 1, dtype=float)
print("初始PR：")
print(pr)

随机跳转概率: 
[[0.9]
 [0.1]]
邻接矩阵: 
[[0. 0. 1. 0. 0.]
 [1. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]]
row_sums: 
[1. 2. 1. 0. 1.]
row_sums处理0->0.1: 
[1.  2.  1.  0.1 1. ]
row_sums[:, np.newaxis]: 
[[1. ]
 [2. ]
 [1. ]
 [0.1]
 [1. ]]
邻接矩阵： 
[[0.  0.  1.  0.  0. ]
 [0.5 0.  0.5 0.  0. ]
 [1.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0. ]
 [0.  1.  0.  0.  0. ]]
初始PR：
[[1. 1. 1. 1. 1.]]


之后，我们就能采用迭代法来计算 PageRank 值。一般我们通过比较每个结点最近两次计算的值是否足够接近，来确定数值是不是已经稳定，以及是不是需要结束迭代。这里为简便起见，我使用了固定次数的循环来实现。如果你的拓扑图比较复杂，需要更多次迭代：

In [8]:
# PageRank算法本身是采用迭代方式进行的，当最终的取值趋于稳定后结束。
for i in range(0, 20):

    # 进行点乘，计算Σ(PR(pj)/L(pj))
    pr = pr.dot(adj)

    # 转置保存Σ(PR(pj)/L(pj))结果的矩阵，并增加长度为N的列向量，其中每个元素的值为1/N，便于下一步的点乘。
    pr_jump = np.full([N, 2], [[0, 1/N]])
    # print("pr_jump:")
    # print(pr_jump)
    
    # 把 “所有行” 的 “第一列” 赋值为 初始 PR
    # 所以这里PR要从行转置成列
    pr_jump[:,:-1] = pr.transpose()
    # print("pr_jump:")
    # print(pr_jump)
    
    # 进行点乘，计算α(Σ(PR(pj)/L(pj))) + (1-α)/N)
    pr = pr_jump.dot(jump)
    # 在本例子中，是 [5,2]的矩阵 x [2,1]的矩阵，结果是 [5, 1] 的矩阵
    # 所以要对结果做转置
    # print("pr:")
    # print(pr)

    # 归一化PageRank得分
    pr = pr.transpose()
    pr = pr / pr.sum()

    print("round", i + 1, pr)

round 1 [[0.37027027 0.24864865 0.37027027 0.00540541 0.00540541]]
round 2 [[0.46740902 0.02498642 0.46740902 0.02009777 0.02009777]]
round 3 [[0.46023676 0.03878962 0.46023676 0.02036842 0.02036842]]
round 4 [[0.46010283 0.03904738 0.46010283 0.02037348 0.02037348]]
round 5 [[0.46010033 0.0390522  0.46010033 0.02037357 0.02037357]]
round 6 [[0.46010028 0.03905229 0.46010028 0.02037357 0.02037357]]
round 7 [[0.46010028 0.03905229 0.46010028 0.02037357 0.02037357]]
round 8 [[0.46010028 0.03905229 0.46010028 0.02037357 0.02037357]]
round 9 [[0.46010028 0.03905229 0.46010028 0.02037357 0.02037357]]
round 10 [[0.46010028 0.03905229 0.46010028 0.02037357 0.02037357]]
round 11 [[0.46010028 0.03905229 0.46010028 0.02037357 0.02037357]]
round 12 [[0.46010028 0.03905229 0.46010028 0.02037357 0.02037357]]
round 13 [[0.46010028 0.03905229 0.46010028 0.02037357 0.02037357]]
round 14 [[0.46010028 0.03905229 0.46010028 0.02037357 0.02037357]]
round 15 [[0.46010028 0.03905229 0.46010028 0.02037357 0.